In [1]:
from base.registry import RUNNER
import torch.nn as nn

@RUNNER.register_module()
class Runner(nn.Module):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, *args, **kwargs):
        print(self.a, self.b)
        return self.a + self.b
    
runner = RUNNER.build(dict(type='Runner', a=1, b=2))

In [1]:
from base.registry import MY_DATASETS
import os.path as osp
root = osp.join('data', 'sonar')
dataset = MY_DATASETS.build(dict(type="WaterTankDataset", root=root))

import torch
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
for output in dataloader:
    print(output.get('img').shape)
    print(output.get('mask').shape)
    break

torch.Size([4, 480, 320])
torch.Size([4, 480, 320])


In [1]:
import torch
import torch.nn as nn
import copy
from mmengine.optim import (OptimWrapper, OptimWrapperDict, _ParamScheduler,
                            build_optim_wrapper)
# from mmengine.registry import OPTIM_WRAPPERS
from tools.runner import Runner

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 3, 3)
        self.bn = nn.BatchNorm2d(3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        return x

model = Model()

optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optim_wrapper = dict(type='OptimWrapper', optimizer=optimizer, clip_grad=None)

optim_wrapper_ = build_optim_wrapper(model, copy.deepcopy(optim_wrapper))
print(optim_wrapper)
Runner._build_optimizer(model, copy.deepcopy(optim_wrapper))

{'type': 'OptimWrapper', 'optimizer': {'type': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0005}, 'clip_grad': None}


AttributeError: 'NoneType' object has no attribute 'pop'

In [ ]:
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optim_wrapper = dict(type='OptimWrapper', optimizer=optimizer, clip_grad=None)
Runner._build_optimizer(model, copy.deepcopy(optim_wrapper))